## Capstone Project - Fun places to visit at Kolkata, India

### Importing of necessary libraries

In [1]:
# importing libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes 
import geopy.geocoders
from geopy.geocoders import Nominatim 
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries are imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

### Connect FourSquare API to extract the details

In [11]:
CLIENT_ID = 'R4AWAOKM5TZZ5P4N5WG1NQ0IRHNXWBJTB4ACNP2EXDSJFUKH'
CLIENT_SECRET = 'TQUWXDGULK0LKAICW1HC0INJHHIT5WXTUJ3MH0APKNVIAV4T'
VERSION = '20180605'
LIMIT=5000
radius=100000
address = 'Kolkata'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
query = 'fun'
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, query, LIMIT, radius)
results = requests.get(url).json()
results

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


{'meta': {'code': 200, 'requestId': '5bd3d998dd5797073eecc329'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Kolkata',
  'headerFullLocation': 'Kolkata',
  'headerLocationGranularity': 'city',
  'query': 'fun',
  'totalResults': 34,
  'suggestedBounds': {'ne': {'lat': 22.612745945000043,
    'lng': 88.39624291117286},
   'sw': {'lat': 22.522745854999954, 'lng': 88.29896168882715}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bc9c62b511f9521982faec7',
       'name': 'Eden Garden',
       'location': {'address': 'BBD Bagh',
        'lat': 22.564541597977065,
        'lng': 88.34329605102539,
        'labeledLatLngs': [{'label': 'display',
          'lat': 22.564541597977065,

### List down all the fun places at Kolkata

In [20]:
venues = results['response']['groups'][0]['items']
fun = json_normalize(venues)
columns = ['venue.name','venue.location.address','venue.location.city','venue.location.distance', 'venue.location.lat', 'venue.location.lng']
fun =fun.loc[:, columns]
fun.columns = [col.split(".")[-1] for col in fun.columns]
fun.city.fillna('Kolkata', inplace=True)
fun.address.fillna('AJC Bose Road', inplace=True)
fun.head()

,name,address,city,distance,lat,lng
0,Eden Garden,BBD Bagh,Kolkata,568,22.564542,88.343296
1,Maidan,AJC Bose Road,Kolkata,2016,22.549906,88.344219
2,Princep Ghat,Strand Rd,Kolkata,1768,22.559267,88.333057
3,Victoria Memorial,"1, Queen's Way",Kolkata,2485,22.545844,88.342890
4,Someplace Else,The Park,Kolkata,1564,22.554250,88.351832


### Representation of all the places at Kolkata on the map

In [14]:
kolkata_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(fun['lat'], fun['lng'], fun['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(kolkata_map)  
    
kolkata_map

### Deploying K-means Algorithm for clustering of places around Kolkata

In [15]:
#Run k-means 

# set number of clusters
kclusters = 4

grouped_clustering = fun.drop(['name','address','city','lat','lng'],axis=1,inplace=False)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# create a new dataframe that includes the cluster as well .
kolkata_fun_trip_spots = fun

# add clustering labels
kolkata_fun_trip_spots['Cluster Labels'] = kmeans.labels_
kolkata_fun_trip_spots

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
rainbow = ['#8000ff', '#2adddd', '#000000', '#ff0000']

# add markers to the map
markers_colors = []
for lat, lng, name, postalcode,cluster in zip(fun['lat'], fun['lng'], fun['name'], fun['city'], fun['Cluster Labels']):
    label = folium.Popup(str(name) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

### Cluster details

#### Cluster 0

In [16]:
kolkata_fun_trip_spots.loc[kolkata_fun_trip_spots['Cluster Labels'] == 0, kolkata_fun_trip_spots.columns[[0] + list(range(2, kolkata_fun_trip_spots.shape[1]))]]

,name,city,distance,lat,lng,Cluster Labels
0,Eden Garden,Kolkata,568,22.564542,88.343296,0
5,Netaji Indoor Stadium,Kolkata,632,22.566157,88.341697,0
8,Indian Museum,Kolkata,1135,22.558022,88.350935,0
13,Millennium Park,Kolkata,615,22.572481,88.344517,0
22,The Cricket Association Of Bengal,Kolkata,423,22.564001,88.346870,0
24,Hind INOX,Kolkata,1228,22.564914,88.359152,0


#### Cluster 1 

In [7]:
kolkata_fun_trip_spots.loc[kolkata_fun_trip_spots['Cluster Labels'] == 1, kolkata_fun_trip_spots.columns[[0] + list(range(2, kolkata_fun_trip_spots.shape[1]))]]

,name,city,distance,lat,lng,Cluster Labels
3,Victoria Memorial,Kolkata,2485,22.545844,88.342890,1
6,Academy of Fine Arts,Kolkata,2735,22.543275,88.345138,1
7,Inox,Kolkata,3284,22.538450,88.351380,1
10,M. P. Birla Planetarium,Kolkata,2464,22.545613,88.346893,1
11,Nandan,Kolkata,2870,22.542034,88.345440,1
14,Rabindra Sadan,Kolkata,2886,22.541941,88.344841,1
15,Kala Mandir,Kolkata,2918,22.543973,88.359582,1
17,Girish Park,Kolkata,2544,22.586069,88.362404,1
21,PVR Cinemas,Hāora,2507,22.563216,88.323704,1
28,Victoria Memorial Grounds,Kolkata,2582,22.585984,88.363124,1


#### Cluster 2

In [17]:
kolkata_fun_trip_spots.loc[kolkata_fun_trip_spots['Cluster Labels'] == 2, kolkata_fun_trip_spots.columns[[0] + list(range(2, kolkata_fun_trip_spots.shape[1]))]]

,name,city,distance,lat,lng,Cluster Labels
16,Star Theatre,Kolkata,3824,22.594419,88.371052,2
18,Park Circus Maidan,Kolkata,3507,22.542981,88.368702,2
19,Inox,Kolkata,3680,22.539139,88.365559,2
23,Zoological Garden,Kolkata,3781,22.536602,88.332905,2
30,Kumartuli,Kolkata,3959,22.600523,88.362569,2
31,Birla Industrial and Technological Museum,Kolkata,4067,22.534400,88.363788,2
32,Harish Park,Kolkata,4166,22.530499,88.343569,2
33,London Paris,Kolkata,5000,22.526372,88.366538,2


#### Cluster 3 

In [9]:
kolkata_fun_trip_spots.loc[kolkata_fun_trip_spots['Cluster Labels'] == 3, kolkata_fun_trip_spots.columns[[0] + list(range(2, kolkata_fun_trip_spots.shape[1]))]]

,name,city,distance,lat,lng,Cluster Labels
1,Maidan,Kolkata,2016,22.549906,88.344219,3
2,Princep Ghat,Kolkata,1768,22.559267,88.333057,3
4,Someplace Else,Kolkata,1564,22.554250,88.351832,3
9,Fort William,Kolkata,1870,22.554237,88.336773,3
12,Marble Palace,Kolkata,2085,22.582444,88.360185,3
20,James Princep Memorial Monument,Kolkata,2140,22.555617,88.331439,3
25,Md. Ali Park,Kolkata,1731,22.577500,88.360720,3
26,Santosh Mitra Square,Kolkata,1872,22.566008,88.365719,3
27,Rabindra Bharati Museum,Kolkata,2253,22.585035,88.359013,3
